# Critical Peak Power Events Notebooks: Exploring how EV charging aligns with Texas's critical peak power events by homes

<p>We will be using data from ERCOT's 4CP calculations to determine how residential homes EV charging habits align with those Peak power events.<br><br>
ERCOT 4CP data is pulled from http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey</p>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
import sys
sys.executable  # shows you your path to the python you're using

In [ ]:
%matplotlib inline

In [ ]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("./config/config.txt")

In [ ]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))

In [ ]:
# These are the ERCOT 4CP events for 2016 - 2019 acquired from
# http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey

event_start_dates = ['2019-06-19 17:00:00-05', '2019-07-30 16:30:00-05', '2019-08-12 17:00:00-05', '2019-09-06 16:45:00-05',
               '2018-06-27 17:00:00-05', '2018-07-19 17:00:00-05', '2018-08-23 16:45:00-05', '2018-09-19 16:30:00-05',
               '2017-06-23 16:45:00-05', '2017-07-28 17:00:00-05', '2017-08-16 17:00:00-05', '2017-09-20 16:45:00-05',
               '2016-06-15 17:00:00-05', '2016-07-14 16:00:00-05', '2016-08-11 16:30:00-05', '2016-09-19 16:15:00-05'
              ]
event_end_dates = ['2019-06-19 17:15:00-05', '2019-07-30 16:45:00-05', '2019-08-12 17:15:00-05', '2019-09-06 17:00:00-05',
               '2018-06-27 17:15:00-05', '2018-07-19 17:15:00-05', '2018-08-23 17:00:00-05', '2018-09-19 16:45:00-05',
               '2017-06-23 17:00:00-05', '2017-07-28 17:15:00-05', '2017-08-16 17:15:00-05', '2017-09-20 17:00:00-05',
               '2016-06-15 17:15:00-05', '2016-07-14 16:15:00-05', '2016-08-11 16:45:00-05', '2016-09-19 16:30:00-05']

In [ ]:
#
#

query = """select dataid, car1 from other_datasets.metadata 
where car1 is not null 
and grid is not null
and egauge_1min_min_time < '2016-06-15'
and egauge_1min_max_time > '2019-09-06'
LIMIT 25
"""

# create a dataframe with the data from the sql query
df = pd.read_sql_query(sqla.text(query), engine)
df

In [ ]:
# grab dataids and convert them to a string to put into the SQL Query
dataids_list = df['dataid'].tolist()
dataids_list
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

In [ ]:
# Assemble the SQL query to pull the data for the selected dataids

first_start = event_start_dates.pop(0)
first_end = event_end_dates.pop(0)
query_2 = """
SELECT dataid, localminute, car1, solar, grid FROM electricity.eg_realpower_1min
WHERE ((localminute >= '{}' AND localminute <= '{}') """.format(first_start, first_end)

for start, end in zip(event_start_dates, event_end_dates):
    query_2 =  query_2 + "OR (localminute >= '{}' AND localminute <= '{}') ".format(start, end)
    
query_2 = query_2 + """ ) AND dataid in ({})""".format(dataids_str)

# here's what that query is
print("sql query is \n" + query_2)

# create a dataframe with the data from the sql query
df2 = pd.read_sql_query(sqla.text(query_2), engine)


df2.head(15)


In [ ]:
df2['localminute'] = df2['localminute'].dt.date
df2.head(10)

In [ ]:
#create new dataframe with columns dataid,car1 and date.
ev_cp_data = pd.DataFrame(df2, columns = ['dataid','car1', 'localminute'])
ev_cp_data.head(10)

In [ ]:
#group data by dataid and date and take average ofcar1. 
#This gives us avaerage ev usage for each home for each CP event day.
ev_cp_data_grouped = ev_cp_data.groupby(['localminute','dataid']).mean()
reset_ev_cp_data_grouped = ev_cp_data_grouped.reset_index()
reset_ev_cp_data_grouped.head(10)

In [ ]:
#filtering data only for those interval when CAR1 is charging..
#todo Check this query

ev_on = reset_ev_cp_data_grouped[reset_ev_cp_data_grouped['car1'] > 0.1]
ev_on = (ev_on['localminute'].value_counts()/50)*100
ev_on

# Observations:

From the above results we can conclude that at least 2% of homes were charging an electric vehicle for each CP event from 2016-2019.